In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# hyoer paramenters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [4]:
# dataset has PIL image so we have to transform it
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

# datasets

train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data', train=True, transform=transform, download=True)

# data loader

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
classes  = ('plan', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
# implenment convnet
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5) # flattern
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [7]:
model = ConvNet().to(device)

In [8]:
# loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [10]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # orignaal shape [4, 3, 32, 32] = 4, 3, 1024
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('finish training')

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
          label = labels[i]
          pred = predicted[i]
          if(label == pred):
            n_class_correct[label] += 1
          n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network : {acc} %')

    for i in range(10):
      acc = 100.0 * n_class_correct[i] / n_class_samples[i]
      print(f'Accuracy of {classes[i]} : {acc} %')


Epoch [1/4], Step [2000/12500], Loss: 1.7544
Epoch [1/4], Step [4000/12500], Loss: 1.9869
Epoch [1/4], Step [6000/12500], Loss: 1.1922
Epoch [1/4], Step [8000/12500], Loss: 1.8317
Epoch [1/4], Step [10000/12500], Loss: 0.7314
Epoch [1/4], Step [12000/12500], Loss: 0.3850
Epoch [2/4], Step [2000/12500], Loss: 1.7601
Epoch [2/4], Step [4000/12500], Loss: 1.2057
Epoch [2/4], Step [6000/12500], Loss: 1.0707
Epoch [2/4], Step [8000/12500], Loss: 1.0874
Epoch [2/4], Step [10000/12500], Loss: 1.6605
Epoch [2/4], Step [12000/12500], Loss: 0.8912
Epoch [3/4], Step [2000/12500], Loss: 1.2028
Epoch [3/4], Step [4000/12500], Loss: 1.6589
Epoch [3/4], Step [6000/12500], Loss: 0.5511
Epoch [3/4], Step [8000/12500], Loss: 0.3829
Epoch [3/4], Step [10000/12500], Loss: 0.9722
Epoch [3/4], Step [12000/12500], Loss: 1.4527
Epoch [4/4], Step [2000/12500], Loss: 0.7889
Epoch [4/4], Step [4000/12500], Loss: 0.7636
Epoch [4/4], Step [6000/12500], Loss: 1.4348
Epoch [4/4], Step [8000/12500], Loss: 1.7219
Epoc